#0. 프로젝트 준비

In [ ]:
# konlyp, mecab , kss install
# 설치 후 런타임 재시작
!pip install konlpy
!curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x
#!pip install kss

In [1]:
# 사용할 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

import math
import time
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 데이터 불러오기



In [ ]:
## csv파일 불러오기
base_path = '/content/drive/MyDrive/nlp미니프로젝트/'
data = pd.read_csv(base_path + 'm0804.csv', delimiter=',', header=0)
# 불필요한 컬럼 제거
data = data.drop(['Unnamed: 0'],axis=1)
data.tail(3)

,가사
31499,현아 날 버리지 말아요 현아 날 떠나지 말아요 현아 당신 떠나시면 이 슬픔 정말 어...
31500,그때는 믿지 않았지 웃어버렸지 사랑하면서도 헤어져야 한다는 말에 아직도 믿진 않지만...
31501,단 한번 단 한번밖에 못해도 그래도 널 사랑할 수 있을까 내 전불 다 걸고 내 앞에...


In [ ]:
# train, test 분리
train = data[:round(0.8*len(data))]
test = data[round(0.8*len(data)):]

print('train:',len(train)
      ,'\ntest:',len(test)
      ,'\n-----------------'
      ,'\ntrain + test:',len(train + test)
      ,'\ndata:',len(data))

train: 25202 
test: 6300 
----------------- 
train + test: 31502 
data: 31502


In [ ]:
## 예제 문장 하나 불러오기
example_1 = train.iloc[15, -1]
example_1

'바람에 나 얼어붙어요 빨리 나와줘 기다리게 하지 마요 차가워진 내 손을 잡아줘요 괜히 자꾸 너를 귀찮게 하는 것 같지만 Everyday 나에게 이 겨울은 말이죠 너에게 조금은 더 안길 수 있는 계절 I love you when you hold me I don t feel the cold 니가 걷는 길에 늘 서 있을게 I love you when you hold me I don t feel the cold 늘 곁에 있어줘 또 봄이 올 때까지 Woo Woo Woo I don t feel the cold Woo Woo Woo I 내 옆에 더 바짝 붙어요 눈이 내려요 감기 걸린 것 같아요 차가워진 나를 좀 안아줘요 괜히 자꾸 너를 귀찮게 하는 것 같지만 Everyday 나에게 이 겨울은 말이죠 내가 그댈 많이 안아 줄 수 있는 계절 I love you when you hold me I don t feel the cold 니가 걷는 길에 늘 서 있을게 I love you when you hold me I don t feel the cold 늘 곁에 있어줘 또 봄이 올 때까지 얼어붙은 모든 게 다 녹아 내리죠 니가 웃을 때 나를 안아줄 때 니가 걷는 길에 늘 서 있을게 I love you when you hold me I don t feel the cold 늘 곁에 있어줘 또 봄이 올 때까지 Woo Woo Woo I don t feel the cold Woo Woo Woo I Woo Woo Woo I don t feel the cold Woo Woo Woo I'

#2. 데이터 전처리
 - 1) 토큰화
 - 2) POS 태깅
 - 3) 불용어 처리



In [ ]:
# csv파일 병합 파일에서 1차적으로 특수문자 제거 완료

# 정규 표현식으로 데이터 정제
# 한글과 공백을 제외하고 모두 제거
def preprocessing(data):
  data['가사'] = data['가사'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")

preprocessing(train)

display(train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,가사
0,그리운 만큼만 기억해 다시 널 만날 수 없게 미안해 붙잡지 않을게 비라도 내리면 좋...
1,바쁘게 지냈었어 밀린 잠도 몰아서 잤어 티빌 켜고 못 봤던 것도 다 보고 약속도 잡...
2,낯선 너를 보면서 지는 해가 서러워 너무 다른 너와 나 여기까지도 힘들었구나 행복이...
3,길을 잃은 작은 아이 시간이 멈춘 그 곳에서 맘을 잃은 작은 아이 너와 걷던 그 거...
4,잘 지내고 있는 거겠죠 좀처럼 연락이 없네요 요즘 많이 바쁜 가 봐요 이미 다른 사...
...,...
25197,사랑 하면 안돼 마음 주면 안돼 불안해 그만해 내가 내게 말해 그대 서있는 곳 반대...
25198,내게 있을 땐 옆에 있는 게 그게 그렇게 소중함을 소중한지 잊는다 결국 잃는다 결국...
25199,닮은 얼굴만 봐도 가슴에 솟구치는 사람 눈을 감고 떠올리면 그리운 사람 발소리만 들...
25200,어떤 사람을 만나는 거니 물을 때 좋은 사람을 만나게 된 거지 대답 해 자랑하고픈 ...


In [ ]:
example_2 = train.iloc[15, -1]
example_2

'바람에 나 얼어붙어요 빨리 나와줘 기다리게 하지 마요 차가워진 내 손을 잡아줘요 괜히 자꾸 너를 귀찮게 하는 것 같지만  나에게 이 겨울은 말이죠 너에게 조금은 더 안길 수 있는 계절              니가 걷는 길에 늘 서 있을게              늘 곁에 있어줘 또 봄이 올 때까지              내 옆에 더 바짝 붙어요 눈이 내려요 감기 걸린 것 같아요 차가워진 나를 좀 안아줘요 괜히 자꾸 너를 귀찮게 하는 것 같지만  나에게 이 겨울은 말이죠 내가 그댈 많이 안아 줄 수 있는 계절              니가 걷는 길에 늘 서 있을게              늘 곁에 있어줘 또 봄이 올 때까지 얼어붙은 모든 게 다 녹아 내리죠 니가 웃을 때 나를 안아줄 때 니가 걷는 길에 늘 서 있을게              늘 곁에 있어줘 또 봄이 올 때까지                          '

## 1) 토큰화

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator # 토큰의 목록 또는 반복자를 받는 내장 팩토리 함수

# #한글 문장 토큰화 하기 위해 필요 (korean sentence splitter)
# import kss

# konlpy에서 라이브러리 불러오기
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Komoran
from konlpy.tag import Okt
from konlpy.tag import Mecab

In [ ]:
# 형태소 분석을 위한 객체 생성
# 꼬꼬마
#kkma = Kkma()

# 코모란
#komoran = Komoran()

# # 한나눔
#hannanum = Hannanum()   # 너무 느려서 사용 안함

# Okt
#okt = Okt()

# Mecab                    
mecab = Mecab()

In [ ]:
# 토큰화 함수 : 토큰 타입을 변경하고 싶으면 return 부분 수정  
# 품사 태깅 퀄리티: 꼬꼬마 > 코모란 > Okt            # .morphs는 형태소만 추출   .pos는 형태소 + 품사 모두 추출   .nouns는 명사 추출
mecab.morphs(train['가사'][4])

In [ ]:
# 토큰화된 가사 저장
token_1 = []

# 형태소만 추출하는 .morphs 형태소 분석기
for sentence in tqdm(train['가사']):
  try:
    # 토큰화
    temp = mecab.morphs(sentence)
    token_1.append(temp)
  except:
    token_1.append('x')
token_1

## 2) POS 태깅

In [ ]:
temp = mecab.pos(sentence)
temp

In [ ]:
# 토큰화된 가사 저장
token = []

# 형태소 + 품사까지 추출하는 .pos 형태소 분석기
for sentence in tqdm(train['가사']):
  # 토큰화
  temp = mecab.pos(sentence)
  token.append(temp)

token

## 3) 불용어 처리

In [ ]:
stopword = []

for i in range(len(token)):
  for j in range(len(token[i])):
    # 튜플 형태의 토큰을 'word'와 'tag'로 언패킹
    word, tag = token[i][j]
    # Mecab 품사 태그 기준
    # SF, SP, SS, SE, SO, SW: 각종 부호들, SL: 외국어, SH: 한자, SN: 숫자
    if tag == 'SF' or tag == 'SP' or tag == 'SS' or tag == 'SE' or tag == 'SO' or tag == 'SW' or tag == 'SL' or tag == 'SH' or tag == 'SN':
      stopword.append(word)

display(stopword)

# 애초에 잘 정제 된 노래 가사이다 보니 특수 기호, 부호들이 없었다.
# 그나마 영어 가사 마저도 데이터 전처리 단계에서 날려버렸..

# 추후에 회의를 통해 추가할 품사가 있는지 검토

[]

In [ ]:
## 불용어 처리 된 테깅
# 토큰화된 가사 저장
tagging_with_sw = []

# 불용어가 추가된 형태소 분석기
for sentence in tqdm(train['가사']):
  # 토큰화
  temp = mecab.morphs(sentence)
  # 불용어 제거
  temp = [word for word in temp if word not in stopword] 
  tagging_with_sw.append(temp)


train['불용어처리 된 가사 토큰'] = tagging_with_sw
train

100%|██████████| 25202/25202 [00:35<00:00, 713.76it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,가사,불용어처리 된 가사 토큰
0,그리운 만큼만 기억해 다시 널 만날 수 없게 미안해 붙잡지 않을게 비라도 내리면 좋...,"[그리운, 만큼, 만, 기억, 해, 다시, 널, 만날, 수, 없, 게, 미안, 해,..."
1,바쁘게 지냈었어 밀린 잠도 몰아서 잤어 티빌 켜고 못 봤던 것도 다 보고 약속도 잡...,"[바쁘, 게, 지냈었, 어, 밀린, 잠, 도, 몰, 아서, 잤, 어, 티, 빌, 켜..."
2,낯선 너를 보면서 지는 해가 서러워 너무 다른 너와 나 여기까지도 힘들었구나 행복이...,"[낯선, 너, 를, 보, 면서, 지, 는, 해, 가, 서러워, 너무, 다른, 너, ..."
3,길을 잃은 작은 아이 시간이 멈춘 그 곳에서 맘을 잃은 작은 아이 너와 걷던 그 거...,"[길, 을, 잃, 은, 작, 은, 아이, 시간, 이, 멈춘, 그, 곳, 에서, 맘,..."
4,잘 지내고 있는 거겠죠 좀처럼 연락이 없네요 요즘 많이 바쁜 가 봐요 이미 다른 사...,"[잘, 지내, 고, 있, 는, 거, 겠, 죠, 좀처럼, 연락, 이, 없, 네요, 요..."
...,...,...
25197,사랑 하면 안돼 마음 주면 안돼 불안해 그만해 내가 내게 말해 그대 서있는 곳 반대...,"[사랑, 하, 면, 안, 돼, 마음, 주, 면, 안, 돼, 불안, 해, 그만, 해,..."
25198,내게 있을 땐 옆에 있는 게 그게 그렇게 소중함을 소중한지 잊는다 결국 잃는다 결국...,"[내게, 있, 을, 땐, 옆, 에, 있, 는, 게, 그게, 그렇게, 소중, 함, 을..."
25199,닮은 얼굴만 봐도 가슴에 솟구치는 사람 눈을 감고 떠올리면 그리운 사람 발소리만 들...,"[닮, 은, 얼굴, 만, 봐도, 가슴, 에, 솟구치, 는, 사람, 눈, 을, 감, ..."
25200,어떤 사람을 만나는 거니 물을 때 좋은 사람을 만나게 된 거지 대답 해 자랑하고픈 ...,"[어떤, 사람, 을, 만나, 는, 거, 니, 물, 을, 때, 좋, 은, 사람, 을,..."


# 3. 감정 사전 구축
 - Word2Vec을 사용한 단어 학습 (CBOW 사용)



In [ ]:
from gensim.models.word2vec import Word2Vec as wv


model_mecab = wv(tagging_with_sw,    # 테깅 데이터를 담은 변수
                 min_count = 3,      # 데이터에 등장하는 단어의 최소 빈도 수 제한
                 size = 128,         # 벡터 스페이스(차원 크기) 설정 (100~300 정도 추천)
                 iter = 20,          # 학습 반복 횟수 (= 딥러닝의 epoch)
                 sg = 0,             # 0이면 CBOW, 1이면 skip-gram 사용
                 batch_words = 1024, # 사전을 구축할 때 한번에 읽을 단어 수
                 window = 5,         # 모델 학습 시 앞뒤로 읽을 단어 수
                 alpha = 0.001,      # learning rate  *디폴트 값(0.0025) 추천
                 negative = 5        # negative samples의 개수 *디폴트 값 추천(클수록 학습속도 느려짐, 너무 작으면 학습 X)
                 )

In [ ]:
# 입력된 단어와 가장 유사한 단어 추출
# x: 알고싶은 단어, y: 검색 개수
def similarity(x,y):
  #유사 단어 검색 및 추가
  vocab = model_mecab.most_similar(x, topn = y) # topn 추출할 유사 단어 개수 top n

  similarity_word = pd.DataFrame(vocab)
  similarity_word.columns = ['유사 단어','유사도']

  display(similarity_word)
  print(f'ㄴ "{x}"와(과) 유사한 단어')

In [ ]:
similarity('짝사랑',10) # '알고싶은 단어', 알고싶은 단어 개수 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


,유사 단어,유사도
0,주제,0.837587
1,어설픈,0.834812
2,지금껏,0.822705
3,실은,0.819760
4,되려,0.817742
5,콜라,0.816146
6,아껴둔,0.814105
7,작곡,0.811623
8,작사,0.811186
9,짜리,0.808480


ㄴ "짝사랑"와(과) 유사한 단어


< 유사단어 정리 표 >

||사랑|행복|-|-|-|
|--|--|--|--|--|--|
|1|행복	0.757362|사랑	0.755427|||
|2|미안	0.725209|소중	0.731544|||
|3|이별	0.713919|약속	0.692954||
|4|후회	0.712072|미안	0.680596|||
|5|말	0.655525|함께	0.674036||
|6|이해	0.651044|부족	0.664977|||
|7|약속	0.636339|감사	0.652247|||
|8|이젠	0.635744|충분	0.642464|||
|9|용서	0.633285|이해	0.639243|||
|10|정말	0.632579|후회	0.629614|||

> 입력한 파라미터
 - CBOW (sg = 0)
 - min_count = 3    
 - size = 128      
 - iter = 20
 - batch_words = 1024
 - window = 5
 - alpha = 0.001
 - negative = 5


In [ ]:
# 감정 사전 함수
def emotion_dictionary(lyric_emotion):
  global resembling_word
  # 사전 구축 알고리즘
  resembling_word = []

  for i in range(len(lyric_emotion)):
    # lyric_emotion 중 선택된 감정 1개의 연관 단어가 저장될 리스트
    selected_emotion = []

    for word in model_mecab.most_similar(lyric_emotion[i],topn = 50): # 필요에 따라 topn 설정하기!
      if(word[1] >= 0.5): # word[1]은 유사도 부분 , 유사도 0.5 이상만 저장
        selected_emotion.append(word)

    resembling_word.append(selected_emotion)

  return resembling_word

In [ ]:
lyric_emotion = ['사랑', '이별']
# 예시 1:'짝사랑','데이트'
# 예시 2: '사랑', '이별'

emotion_dictionary(lyric_emotion)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # This is added back by InteractiveShellApp.init_path()


[[('행복', 0.736953616142273),
  ('미안', 0.7002512216567993),
  ('이별', 0.680718719959259),
  ('후회', 0.6741137504577637),
  ('말', 0.6436918377876282),
  ('이해', 0.6392656564712524),
  ('정말', 0.6130786538124084),
  ('용서', 0.6110212206840515),
  ('약속', 0.6074888706207275),
  ('사람', 0.5997127890586853),
  ('아프', 0.5988306403160095),
  ('나', 0.5955733060836792),
  ('부족', 0.5790762305259705),
  ('마지막', 0.573980450630188),
  ('이젠', 0.5708649158477783),
  ('믿', 0.5678460597991943),
  ('잊', 0.5662732720375061),
  ('이렇게', 0.5624731183052063),
  ('고백', 0.5569831132888794),
  ('어떡', 0.5528459548950195),
  ('난', 0.5432081818580627),
  ('기억', 0.5367077589035034),
  ('알', 0.5337257981300354),
  ('소중', 0.5325065851211548),
  ('이제', 0.5317033529281616),
  ('진심', 0.5256588459014893),
  ('아직', 0.5255136489868164),
  ('충분', 0.516739010810852),
  ('시작', 0.5134662389755249),
  ('지금', 0.5133589506149292),
  ('줄', 0.5119651556015015),
  ('미워해야', 0.505484402179718),
  ('원망', 0.5024927854537964),
  ('준비', 0.5001147

# 4. 감정 유사도 계산
 - 단어 간 거리 유사도 판별 (Word2Vec 임베딩 모델 사용)



In [ ]:
# 감정 계산기
def emotion_calculator(data):
  global data_dict
  # 분류를 원하는 감정 수 만큼 data_dict 값 설정!!!          < 중요 >
  data_dict = {lyric_emotion[0]: 0, 
               lyric_emotion[1]: 0,
               #lyric_emotion[2]: 0,
               }

  for i in range(len(lyric_emotion)): # i = [0,1,2,...]
    for j in resembling_word[i]:      # j = [('말', 0.5438262224197388)('이래야', 0.5401289463043213)...]
      for k in range(len(data)):      # k = [0,1,2,3,4,...]
        if j[0] == data[k]:
          # print(j[1]) # 각 토큰에 대한 점수
          data_dict[lyric_emotion[i]] = data_dict[lyric_emotion[i]]+j[1]
  return data_dict

In [ ]:
# 아래 함수에 넣고 싶은 가사 입력! 
# *불용어 처리 완료 된 토큰화 가사를 입력하셔야 됩니다!!
emotion_calculator(train['불용어처리 된 가사 토큰'][3])

{'사랑': 8.947540402412415, '이별': 7.186211824417114}

In [ ]:
# 1순위 감정 계산기
def best_emotion_calculator(data_dict):
  max_num = 0
  best_emotion = []

  for i in data_dict.keys():
    x = data_dict[i]
    y = max_num

    if x >= y:
      max_num = x
      best_emotion.append(i)

  return print(f'해당 곡의 가장 큰 감정:  {best_emotion[-1]}\n점수:  {max_num}')

In [ ]:
best_emotion_calculator(data_dict)

해당 곡의 가장 큰 감정:  사랑
점수:  8.947540402412415


#5. 감정 분류
 - 1) 감정 레이블링
  - 가사의 감정을 더 정확하게 분류하기 위해 순환신경망 모델 LSTM을 활용한 지도학습 진행
  - 지도학습이기에 정답데이터로 사용할 레이블링 작업 필요..
 - 2) 단어 임베딩
 - 3) LSTM을 활용한 감정 분류 모델 구축



In [ ]:
## 레이블링 테스트

# 테스트 할 데이터 불러오기
labeling_test = train[500:1000].copy()
labeling_test

,가사,불용어처리 된 가사 토큰
500,햇살이 좋은 하늘에 고백해 첫눈에 반했던 너를 기억해 내겐 운명처럼 나의 바램처럼 ...,"[햇살, 이, 좋, 은, 하늘, 에, 고백, 해, 첫눈, 에, 반했, 던, 너, 를..."
501,그대에겐 아무렇지 않아도 지금 이 순간 나에게는 꿈만 같아요 이렇게 함께 하는 시간...,"[그대, 에겐, 아무렇, 지, 않, 아도, 지금, 이, 순간, 나, 에게, 는, 꿈..."
502,항상 네 생각이 나서 언제든 보고싶어서 그래서 많이 두려웠어 내 마음 들켜버릴까봐 ...,"[항상, 네, 생각, 이, 나, 서, 언제, 든, 보, 고, 싶, 어서, 그래서, ..."
503,너인 걸 느끼고 있어 너라고 알 수가 있어 사랑이란 그 감정을 알게 해줬던 너를 잊...,"[너, 인, 걸, 느끼, 고, 있, 어, 너, 라고, 알, 수, 가, 있, 어, 사..."
504,그댄 오늘 밤 잘 자요 어깨 위에 그 짐은 덜어버려요 해줄 수 있는 내 허접한 선물...,"[그댄, 오늘, 밤, 잘, 자요, 어깨, 위, 에, 그, 짐, 은, 덜, 어, 버려..."
...,...,...
995,구름 안에 모여있는 빗방울들이 무거워져 하나둘씩 내려와 가만히 감싸주는 소리 들리네...,"[구름, 안, 에, 모여, 있, 는, 빗방울, 들, 이, 무거워져, 하나, 둘, 씩..."
996,산다는 건 거친 바다를 헤쳐가듯 외로움에 긴긴 밤을 지새야 하는 눈물 마르고 아침이...,"[산다는, 건, 거친, 바다, 를, 헤쳐, 가, 듯, 외로움, 에, 긴긴, 밤, 을..."
997,꽤 오래됐어 이 자리 어정쩡한 이 마음도 끊임없이 뭔가 하려 마음먹어도 손가락 하나...,"[꽤, 오래, 됐, 어, 이, 자리, 어정쩡, 한, 이, 마음, 도, 끊임없이, 뭔..."
998,뭐가 그리 어려운 건데 아쉬운 눈빛만 보내지 말아줘 더는 숨길 수가 없잖아 말해줘 ...,"[뭐, 가, 그리, 어려운, 건데, 아쉬운, 눈빛, 만, 보내, 지, 말, 아, 줘..."


In [ ]:
labeling_test['가사'].iloc[10]

'찬 바람이 불어오는 이 거리에 서면 유난히 니가 그리워져 참 따스했던 표정과 니 손길과 둘만의 이야기 혹시 너도 날 생각하니 서로 힘이 들 때 안아주던 그 자리에 서서 기다려 언제까지나 기억해줘 그 때 우리 소중했던 감정들 함께 노래를 부를 수 있기를 기도해줘 시간이 지나도 우리 웃을 수 있기를 바래요 어느 샌가 커져버린 내 맘에 빈자리 또 다시 니가 그리워져 오랜 시간을 함께 걸었던 우리가 만든 추억 하나 간직할 테니 기억해줘 그 때 우리 소중했던 감정들 함께 노래를 부를 수 있기를 기도해줘 시간이 지나도 우리 웃을 수 있기를 바래요 눈물이 흐르도록 아픈 시간도 너 때문에 나 이겨낸거죠 바래왔던 날들 기억의 언덕에 서면 그리운 바람이 불어올까 손잡아줘 아프지만 아련했던 그 날에 함께 노래를 부를 수 있기를 기도해줘 시간이 지나도 우리 웃을 수 있기를 바래요'

In [ ]:
# 사랑:0 이별:1 로 지정
label = [0,0,0,1,0,1,0,0,0]

In [ ]:
# 가사 감정 레이블링

## 논문에 나온 레이블링 과정

result_pd = pd.Series(result_list) # result_list는 아마도 불용어 처리까지 마친 토큰화된 가사 리스트??
data_label = pd.concat([data_temp, result_pd],axis=1)  # 레이블을 입력할 임시 컬럼 'data_temp'와 가사 리스트 하나로 합병
label_df = {"슬픔":0, "부정":1, "무관심":2, "분노":3} # 노가다 시작..
data_label['label'] = data_label.iloc[:,1].map(label_df) # 새로운 컬럼에 레이블 입력
data_label.head(10) # 잘 적용됬는지 확인!

In [ ]:
# 레이블링 작업 끝나면 모든 가사를 빈도 기반 임베딩 작업 진행
# >> 모든 가사를 대상으로 빈도를 측정한 후, 빈도 기반으로 단어를 숫자로 바꾸는 작업 진행
# >> 논문 27p ~ 28p 참고

In [ ]:
#LSTM 모델 학습

# 논문에 나온 알고리즘
model = Sequential()
model.add(LSTM(32, input_shape = (new_Series.shape[0] + 1, 150),return_sequences=True)) 
model.add(LSTM(32)) # 입력 퍼셉트론 수
model.add(Dropout(0.3))
model.add(Dense(30, activation="relu", kernel_initializer='he_normal')) 
model.add(Dense(4, activation='softmax')) 
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

history = model.fit(X_train, Y_train, epochs=1000, 
                    validation_data=(X_test,Y_test), 
                    callbacks=[es,mc], batch_size=X_train.shape[0])


6. 실험 평가
 - 1) Word2Vec 모델 비교
 - 2) 문맥 의미 파악
 - 3) 모델 비교 실험
    - SVM
    - Logistic Regression
    - LSTM
 - 4) 신규 가사 감정 테스트
 - 5) 기존 연구와 비교분

In [ ]:
## test

# 전처리
preprocessing(train)

# train으로 학습 된 유사 단어 추출 모델
similarity('이별',10)

